# SMPL-basic

SMPL is a classic body model for human body shape and pose estimation. We can get joints position and the vertices of mesh from SMPL's results.

In this notebook, we will introduce the basic usage of `smplx.SMPL` model, a rough feeling of the four parameters, and some common problems you may encounter.

And there is a "SMPL family", which is a set of models related to SMPL. You can have a overview of them in [SMPL wiki](https://meshcapade.wiki/SMPL).

## Preparation

Usually, we use three different checkpoints: `SMPL_NEUTRAL.pkl`, `SMPL_MALE.pkl` and `SMPL_FEMALE.pkl`. You can get the first one from [SMPLify > Downloads > "SMPLIFY_CODE_V2.ZIP"](https://smplify.is.tue.mpg.de/download.php) and the other two from [SMPL > Downloads > "Download version 1.0.0 for Python 2.7 (female/male. 10 shape PCs)"](https://smpl.is.tue.mpg.de/download.php). Or you can choose to download them through scripts like [these lines (WHAM's script for example)](https://github.com/yohanshin/WHAM/blob/2b54f7797391c94876848b905ed875b154c4a295/fetch_demo_data.sh#L2-L30).

> If you download the checkpoint from the website yourself, you may get a `.pkl` file named `basicModel_neutral_lbs_10_207_0_v1.0.0.pkl` rather then `SMPL_NEUTRAL.pkl`. Don't be worried, just renamed it.

> The size of `v1.0.0` version body models is usually around 37MB. Those models support only **10** principal components for shape(betas). You can also find the `v1.1.0` version of male, female and neutral body models from [SMPL > Downloads > "Download version 1.1.0 for Python 2.7 (female/male/neutral, 300 shape PCs)"](https://smpl.is.tue.mpg.de/download.php), they are usually in the size of 247MB and support **300** principal components for shape(betas).


After downloading the SMPL's checkpoints, you should put them to `data_inputs/body_models/smpl`, your directory tree should look like this:

```
.
├── SMPL_FEMALE.pkl
├── SMPL_MALE.pkl
└── SMPL_NEUTRAL.pkl
```

## Tutorials

### Environment Preparation

In [1]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL
from pytorch3d import transforms  # You may use this package when performing rotation representation transformation.

# Things you don't need to care about. They are just for driving the tutorials.
from lib.utils.path_manager import PathManager
from lib.viewer.wis3d_utils import HWis3D as Wis3D
from lib.skeleton import Skeleton_SMPL24

pm = PathManager()

### Load SMPL model

SMPL has different models weights for different genders. Make sure you use the correct model for your project.

Usually, we just use the neutral model if we can't access the gender information. 

Here, we will just use neutral model for simplicity. You can try the other genders if you want. You can just re-assign the `body_model` variable in the next cell and re-run the remaining cells.

In [2]:
body_models = {}
genders = ['neutral', 'female', 'male']  # case insensitive

for gender in genders:
    body_models[gender] = SMPL(
            model_path = pm.inputs / 'body_models' / 'smpl',
            gender     = gender,
        )

In [3]:
# Prepare some parameters for later inference.
B = 150
body_model : SMPL = body_models['neutral']  # use neutral for example

# Prepare mesh template for later visualization.
# Tips: mesh = vertices + faces, and the faces are the indices of vertices, which won't change across SMPL's outputs.
mesh_temp : np.ndarray = body_model.faces  # (13776, 3)

### SMPL Inference

In [4]:
# Inference.
smpl_out = body_model(
        betas         = torch.zeros(B, 10),     # shape coefficients
        global_orient = torch.zeros(B, 1, 3),   # axis-angle representation
        body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
        transl        = torch.zeros(B, 3),
    )

# Check output.
joints : torch.Tensor = smpl_out.joints    # (B, 45, 3)
verts  : torch.Tensor = smpl_out.vertices  # (B, 6890, 3)
print(joints.shape, verts.shape)

torch.Size([150, 45, 3]) torch.Size([150, 6890, 3])


### SMPL Skeleton

SMPL has a 24-joint skeleton. We can get the joints position from the SMPL model's output.

> You may notice that there are 45 joints returned by `smplx.SMPL` in total. Here we only refer to the first 24 joints as the skeleton. You can check [SMPL-details#Understand-the-Joints-45](./SMPL_details.ipynb) for more details about this.

The joints' index of SMPL model is shown below. The figure is from the [SMPL-made-simple-FAQs](https://files.is.tue.mpg.de/black/talks/SMPL-made-simple-FAQs.pdf).

![](./assets/SMPL-joints.png)

And the bones (edge in kinematic tree) are shown below.

In [5]:
chains = [
        [ 0,  1,  4,  7, 10],      # left leg
        [ 0,  2,  5,  8, 11],      # right leg
        [ 0,  3,  6,  9, 12, 15],  # spine & head
        [12, 13, 16, 18, 20, 22],  # left arm
        [12, 14, 17, 19, 21, 23],  # right arm
    ]
bones = [
        [ 0,  1], [ 1,  4], [ 4,  7], [ 7, 10],            # left leg
        [ 0,  2], [ 2,  5], [ 5,  8], [ 8, 11],            # right leg
        [ 0,  3], [ 3,  6], [ 6,  9], [ 9, 12], [12, 15],  # spine & head
        [12, 13], [13, 16], [16, 18], [18, 20], [20, 22],  # left arm
        [12, 14], [14, 17], [17, 19], [19, 21], [21, 23],  # right arm
    ]

### SMPL Parameters

Here we will learn the main for SMPL parameters through several demos. The main parameters are:

1. betas
2. global_orient
3. body_pose
4. transl

#### 1. betas | $\beta \in \R^{||\beta||}$

Betas control the shape of the model. Usually we use the default 10 shape coefficients. It depends on the model you load.

You may see this(below) before, that means you are using a model with 10 shape coefficients.

> "WARNING: You are using a SMPL model, with only 10 shape coefficients." 

In [ ]:
def learn_betas(
    selected_component : int = 0,
    lower_bound : int = -5,
    upper_bound : int = +5,
):
    def make_fake_data():
        fake_betas = torch.zeros(B, 10)
        fake_betas[:, selected_component] = torch.linspace(lower_bound, upper_bound, B)
        return fake_betas
    fake_betas = make_fake_data()

    # Inference.
    smpl_out = body_model(
            betas         = fake_betas,             # shape coefficients
            global_orient = torch.zeros(B, 1, 3),   # axis-angle representation
            body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
            transl        = torch.zeros(B, 3),
        )

    # Check output.
    joints : torch.Tensor = smpl_out.joints    # (B, 45, 3)
    verts  : torch.Tensor = smpl_out.vertices  # (B, 6890, 3)
    faces  : np.ndarray   = body_model.faces   # (13776, 3)

    def visualize_results():
        """ This part is to visualize the results. You are supposed to ignore this part. """
        shape_wis3d = Wis3D(
                pm.outputs / 'wis3d',
                'SMPL-parameters-beta',
            )

        shape_wis3d.add_motion_verts(
            verts  = verts,
            name   = f'betas[:, {selected_component}] from {lower_bound} to {upper_bound}',
            offset = 0,
        )
        shape_wis3d.add_motion_mesh(
            verts  = verts,
            faces  = faces,
            name   = f'surface: betas[:, {selected_component}] from {lower_bound} to {upper_bound}',
            offset = 0,
        )
        shape_wis3d.add_motion_skel(
            joints = joints[:, :24],
            bones  = Skeleton_SMPL24.bones,
            colors = Skeleton_SMPL24.bone_colors,
            name   = f'skeleton: betas[:, {selected_component}] from {lower_bound} to {upper_bound}',
            offset = 0,
        )
    visualize_results()

We will visualize the effects of the changes on certain coefficient.

Here, `learn_betas(k)` means we will visualize the SMPL outputs when the k-th coefficient is changed from -5 to +5.

In [11]:
learn_betas(0)
learn_betas(1)
learn_betas(2)

Now start the viewer.

You will see that, the skeleton will mis-align with the mesh when the coefficients are very "sharp".
Some of the coefficient control the height, the length of the limbs, etc.

In [ ]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

#### 2. global_orient | $\theta_r\in\R^3$ (part of $\theta \in \R^{3\times24}$)

Global orient control the face direction of the virtual human, which is also the "rotation" of the root joint.

You may have to check [axis angle](https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation) before going on.
For example, a vector $\vec{r} = [x, y, z]$ represents a rotation around the axis $\frac{\vec{r}}{||\vec{r}||}$ in radians $||\vec{r}||$.

In [ ]:
def learn_orient():
    def make_fake_data():
        fake_orient = torch.zeros(B, 1, 3)
        fake_orient[   : 50, :, 0] = torch.linspace(0, 2 * np.pi, 50).reshape(50, 1)  # about x-axis
        fake_orient[ 50:100, :, 1] = torch.linspace(0, 2 * np.pi, 50).reshape(50, 1)  # about y-axis
        fake_orient[100:150, :, :] = torch.linspace(0, 2 * np.pi, 50).reshape(50, 1, 1).repeat(1, 1, 3)  # about x=y=z
        fake_orient[100:150, :, :] /= np.sqrt(3)  # Ensure the norm is 2pi.
        return fake_orient
    fake_orient = make_fake_data()

    # Inference.
    smpl_out = body_model(
            betas         = torch.zeros(B, 10),     # shape coefficients
            global_orient = fake_orient,            # axis-angle representation
            body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
            transl        = torch.zeros(B, 3),
        )

    # Check output.
    joints : torch.Tensor = smpl_out.joints    # (B, 45, 3)
    verts  : torch.Tensor = smpl_out.vertices  # (B, 6890, 3)
    faces  : np.ndarray   = body_model.faces   # (13776, 3)

    def visualize_results():
        """ This part is to visualize the results. You are supposed to ignore this part. """
        orient_wis3d = Wis3D(
                pm.outputs / 'wis3d',
                'SMPL-parameters-global_orient',
            )

        # Prepare the rotation axis.
        axis_x   = torch.tensor([[0, 0, 0], [3, 0, 0]], dtype=torch.float32)
        axis_y   = torch.tensor([[0, 0, 0], [0, 3, 0]], dtype=torch.float32)
        axis_xyz = torch.tensor([[0, 0, 0], [1, 1, 1]], dtype=torch.float32)
        axis_all = torch.concat(
            [
                axis_x.reshape(1, 2, 3).repeat(50, 1, 1),
                axis_y.reshape(1, 2, 3).repeat(50, 1, 1),
                axis_xyz.reshape(1, 2, 3).repeat(50, 1, 1),
            ], dim = 0)
        axis_all[:, :, :] += joints[:, [0], :] # move the axis to the root joints


        orient_wis3d.add_vec_seq(
            vecs = axis_all,
            name = 'rotation axis',
        )
        orient_wis3d.add_motion_verts(
            verts  = verts,
            name   = f'vertices',
            offset = 0,
        )
        orient_wis3d.add_motion_mesh(
            verts  = verts,
            faces  = faces,
            name   = f'surface',
            offset = 0,
        )
        orient_wis3d.add_motion_skel(
            joints = joints[:, :24],
            bones  = Skeleton_SMPL24.bones,
            colors = Skeleton_SMPL24.bone_colors,
            name   = f'skeleton',
            offset = 0,
        )
    visualize_results()

We will visualize the effects of the changes on `global_orient`.

Here, `learn_orient()` will rotate the digital human in three ways:

1. `fake_orient[  0: 50]` = $[0, 0, 0] \rightarrow [2\pi,  0, 0 ]$, rotation about $x$-axis
2. `fake_orient[ 50:100]` = $[0, 0, 0] \rightarrow [ 0, 2π, 0 ]$, rotation about $y$-axis
3. `fake_orient[100:150]` = $[0, 0, 0] \rightarrow [2π, 2π, 2π]$, rotation about $x=y=z$ axis

You are supposed to make sure you understand the axis-angle representation before going on.

In [17]:
learn_orient()

Now start the Wis3D viewer.

**You will see that, the rotation axis starts from the position of root joint, rather than the origin of the coordinates.**

In [ ]:
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

There is still one thing you should know: the exact value of global orientation is related to the **coordinates** (e.g., camera coordinates, global coordinates) you are using. (So is the translation in SMPL.)

#### 3. body_pose | $\theta_b\in\R^{3\times23}$ (part of $\theta \in \R^{3\times24}$)

You should be sensitive to these numbers combinations: (23, 3), (23, 6), (23, 3, 3), (69,), (24, 3), (24, 6), (24, 3, 3), (72,). The tensor or array with these shapes are usually related to the body pose.

Also, a pose is represented in the way of **kinematic chains**, the `body_pose` provide the **relative rotation of each joint to its parent joint**, and the SMPL model will solve a **forward kinematics** problem to get the final position of each joints, i.e. the final pose.

> Check [this lecture (GAMES 105 Lec3)](https://www.bilibili.com/video/BV1GG4y1p7fF?p=3&vd_source=13807e82155f985591ed6f1b4e3434ed) if you are interested in the topic of forward/inverse kinematic.

Sometimes we will group the `global_orient` and `body_pose` together as a 24 "joints" `pose`, and the `global_orient` is always the first element of this `pose`.

Although in SMPL, the joint rotation is represented in the form of **axis-angle**, people are more likely to use an **6D-rotation** representation extracted from the 3x3 rotation matrix for a network to train. We will dive into this problem in another notebook.

In [ ]:
def learn_body_pose(eg_path):
    def load_eg_params(eg_path):
        eg_params = np.load(eg_path, allow_pickle=True).item()
        eg_body_pose_aa = torch.from_numpy(eg_params['body_pose'])  # (1, 23, 3)
        return eg_body_pose_aa

    def make_fake_data():
        tgt_body_pose = load_eg_params(eg_path)  # (1, 23, 3)
        grad_weights = torch.linspace(0, 1, B).reshape(B, 1, 1)  # (B, 1, 1)
        fake_body_pose = grad_weights * tgt_body_pose  # (B, 23, 3)
        return fake_body_pose

    fake_body_pose = make_fake_data()  # (B, 23, 3)

    # Inference.
    smpl_out = body_model(
            betas         = torch.zeros(B, 10),    # shape coefficients
            global_orient = torch.zeros(B, 1, 3),  # axis-angle representation
            body_pose     = fake_body_pose,        # axis-angle representation
            transl        = torch.zeros(B, 3),
        )

    # Check output.
    joints : torch.Tensor = smpl_out.joints    # (B, 45, 3)
    verts  : torch.Tensor = smpl_out.vertices  # (B, 6890, 3)
    faces  : np.ndarray   = body_model.faces   # (13776, 3)

    def visualize_results():
        """ This part is to visualize the results. You are supposed to ignore this part. """
        orient_wis3d = Wis3D(
                pm.outputs / 'wis3d',
                'SMPL-parameters-body_pose',
            )

        orient_wis3d.add_motion_verts(
            verts  = verts,
            name   = f'vertices',
            offset = 0,
        )
        orient_wis3d.add_motion_mesh(
            verts  = verts,
            faces  = faces,
            name   = f'surface',
            offset = 0,
        )
        orient_wis3d.add_motion_skel(
            joints = joints[:, :24],
            bones  = Skeleton_SMPL24.bones,
            colors = Skeleton_SMPL24.bone_colors,
            name   = f'skeleton',
            offset = 0,
        )
    visualize_results()

In [24]:
learn_body_pose(pm.inputs / 'examples/ballerina.npy')

In [ ]:
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host

#### 4. transl | $\Gamma\in\R^{3}$

Translation control the position of the virtual human in the 3D space. In camera coordinates, the translation usually represent the distance between the camera and the human. In global coordinates, the translation usually has the similar meaning as the movement of the human.

We also use the word "trajectory" to represent the historical position of the root joint. Sometimes, the "trajectory" will be projected to the ground plane. Remember, the specific meaning of the translation is related to the specific work.

In [27]:
def learn_transl(rotation:bool = False):
    def make_fake_data():
        phase = torch.arange(50) / 50.0 * (2 * np.pi)  # 0 ~ 2𝜋

        # Generate fake translation.
        fake_transl = torch.zeros(B, 3)
        # Part 1, [0, 50)
        fake_transl[   : 25, 2] = torch.sin(phase[:25])  # along z-axis
        fake_transl[ 25: 50, 1] = torch.sin(phase[:25])  # along y-axis
        # Part 2, [50, 75) + [75, 100)
        fake_transl[ 50:100, 1] = torch.sin(phase)       # along y-axis
        fake_transl[ 50: 75, 0] = torch.sin(phase[::2])  # along y-axis
        fake_transl[ 75:100, 2] = torch.sin(phase[::2])  # along y-axis
        # Part 3, [100, 150)
        fake_transl[100:150, 0] = torch.cos(phase) * phase / (2 * np.pi)
        fake_transl[100:150, 2] = torch.sin(phase) * phase / (2 * np.pi)

        # Generate fake rotation (if needed).
        fake_orient = torch.zeros(B, 1, 3)
        if rotation:
            fake_orient[:, :, 1] = torch.linspace(0, 3 * (2 * np.pi), B).reshape(B, 1)  # about y-axis

        return fake_transl, fake_orient

    fake_transl, fake_orient = make_fake_data()

    # Inference.
    smpl_out = body_model(
            betas         = torch.zeros(B, 10),     # shape coefficients
            global_orient = fake_orient,            # axis-angle representation
            body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
            transl        = fake_transl,
        )

    # Check output.
    joints : torch.Tensor = smpl_out.joints    # (B, 45, 3)
    verts  : torch.Tensor = smpl_out.vertices  # (B, 6890, 3)
    faces  : np.ndarray   = body_model.faces   # (13776, 3)

    def visualize_results():
        """ This part is to visualize the results. You are supposed to ignore this part. """
        transl_wis3d = Wis3D(
                pm.outputs / 'wis3d',
                'SMPL-parameters-transl',
            )

        transl_wis3d.add_traj(
            positions = fake_transl,
            name      = f'trajectory (rotating)' if rotation else 'trajectory',
            offset    = 0,
        )
        transl_wis3d.add_motion_verts(
            verts  = verts,
            name   = f'vertices (rotating)' if rotation else 'vertices',
            offset = 0,
        )
        transl_wis3d.add_motion_mesh(
            verts  = verts,
            faces  = faces,
            name   = f'surface (rotating)' if rotation else 'surface',
            offset = 0,
        )
        transl_wis3d.add_motion_skel(
            joints = joints[:, :24],
            bones  = Skeleton_SMPL24.bones,
            colors = Skeleton_SMPL24.bone_colors,
            name   = f'skeleton (rotating)' if rotation else 'skeleton',
            offset = 0,
        )
    visualize_results()

We will visualize the effects of the changes on `transl`.

Here, `learn_transl()` will make the digital human moves. Please check the code yourself and match the lines with the movements in the visualization results.

In [28]:
learn_transl(rotation=False)
learn_transl(rotation=True)

Now start the Wis3D viewer.

There are two things you should notice:

1. The `transl` is defined in a static coordinate (compared to the ego coordinate of the agent). So the orientation changes will not affect the translation. Check the visualization results, you will find the trajectory of the rotating human and the non-rotating human are the same.
2. The position of the root joint has small differences with the `transl` in the SMPL model, as the root joint won't be put on the origin of the coordinates in zero-pose.

In [ ]:
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

## References

- [SMPL Project Page](https://smpl.is.tue.mpg.de/)
- [SMPL-made-simple-FAQs](https://files.is.tue.mpg.de/black/talks/SMPL-made-simple-FAQs.pdf)
- [SMPL wiki](https://meshcapade.wiki/SMPL)